In [1]:
############LIBRARIES################
import sys
sys.path.append("..\\..\\CreateNERSilverSet_NewPipelineOutput")
sys.path.append("..\\..\\PrepareBERT_forDatabricks")

from NERSilverSetFromSentences import *
from GrantAndOrganizationMentionsFromGoldSet import grant_and_organization_mentions_from_gold_set
from AnnotateForBert import *

import pandas as pd
import numpy as np
from transformers import BertTokenizerFast
import pickle
import random
import torch

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

In [2]:
############VARIABLES################
#Load tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

C:\Users\aydxng\Anaconda3\envs\Python37\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\aydxng\Anaconda3\envs\Python37\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [3]:
######READ THE DATA##########
path = "C:\\Users\\aydxng\\Documents\\ds-fundingbodies-linkingcomponent-masterthesis\\Thesis\\CreateNERSilverSet_NewPipelineOutput\\"
with open(path+'df_sent_divided.pkl','rb') as f:
    df= pickle.load(f)
    
#Discard the samples in "Validation" folder
folders_validation = ['Validation\\Round_2','Validation\\Round_3','Validation\\Round_6','Validation\\Round_7','Validation\\Round_9',
                      'Validation\\Round_11','Validation\\Round_13','Validation\\Round_17','Validation\\Round_18','Validation\\Round_19',
                      'Validation\\Round_20','Validation\\Round_22','Validation\\Round_23','Validation\\Round_25','Validation\\Round_27',
                      'Validation\\Round_28','Validation\\Round_30','Validation\\Round_32']
df = df[~df.Folder.isin(folders_validation)].copy(deep=True)

#Split train/valid/test
train = df[df.Dataset=='Training'].reset_index(drop=True).copy(deep=True)
valid = df[df.Dataset=='Validation'].reset_index(drop=True).copy(deep=True)

#Drop not-needed columns
train.drop(['Dataset','Pipeline_Span_Tags_IOB','Pipeline_Span_Tags_GRT_IOB','Pipeline_Span_Tags_ORG_IOB'],
       axis=1, inplace=True)
valid.drop(['Dataset','Pipeline_Span_Tags_IOB','Pipeline_Span_Tags_GRT_IOB','Pipeline_Span_Tags_ORG_IOB'],
       axis=1, inplace=True)
del df

In [4]:
######GET GOLD NAMED ENTITIES##########
path_to_gold = "C:\\Users\\aydxng\\OneDrive - Reed Elsevier Group ICO Reed Elsevier Inc\\Desktop\\ElsevierData\\Dataset2020July\\Dataset2020July\\"
folders_kpi = ['GoldSet_2019\\KPI\\']
folders_trainin_train = ['GoldSet_2019\\Trainin-Train\\']
folders_trainin_validation = ['GoldSet_2019\\Trainin-Validation\\']
folders_trainin_test = ['GoldSet_2019\\Trainin-Test\\']
folders_regression = ['GoldSet_2019\\Regression\\']
folders_improvement = ['Improvement\\Round_1\\','Improvement\\Round_10\\','Improvement\\Round_12\\','Improvement\\Round_14\\','Improvement\\Round_15\\','Improvement\\Round_21\\','Improvement\\Round_24\\','Improvement\\Round_26\\','Improvement\\Round_29\\','Improvement\\Round_31\\','Improvement\\Round_4\\','Improvement\\Round_5\\','Improvement\\Round_8\\' ]
#Gold named entities per folder
org_dict_kpi, grant_dict_kpi = grant_and_organization_mentions_from_gold_set([path_to_gold+x for x in folders_kpi])
org_dict_trainin_train, grant_dict_trainin_train = grant_and_organization_mentions_from_gold_set([path_to_gold+x for x in folders_trainin_train])
org_dict_trainin_validation, grant_dict_trainin_validation = grant_and_organization_mentions_from_gold_set([path_to_gold+x for x in folders_trainin_validation])
org_dict_trainin_test, grant_dict_trainin_test = grant_and_organization_mentions_from_gold_set([path_to_gold+x for x in folders_trainin_test])
org_dict_regression, grant_dict_regression = grant_and_organization_mentions_from_gold_set([path_to_gold+x for x in folders_regression])
org_dict_improvement, grant_dict_improvement = grant_and_organization_mentions_from_gold_set([path_to_gold+x for x in folders_improvement])
org_dict_gold = dict()
grant_dict_gold = dict()
org_dict_gold.update(org_dict_kpi)
grant_dict_gold.update(grant_dict_kpi)
org_dict_gold.update(org_dict_trainin_train)
grant_dict_gold.update(grant_dict_trainin_train)
org_dict_gold.update(org_dict_trainin_validation)
grant_dict_gold.update(grant_dict_trainin_validation)
org_dict_gold.update(org_dict_trainin_test)
grant_dict_gold.update(grant_dict_trainin_test)
org_dict_gold.update(org_dict_regression)
grant_dict_gold.update(grant_dict_regression)
org_dict_gold.update(org_dict_improvement)
grant_dict_gold.update(grant_dict_improvement)

Error in file:  00812K_622654249.ann
Error in file:  00901K_618203602.ann
Error in file:  07147K_618282364.ann
Error in file:  09209K_358501993.ann
Error in file:  06041T_618636215.ann
Error in file:  06821T_620696511.ann
Error in file:  07347T_621231937.ann
Error in file:  07451R_605779630.ann
Error in file:  07964R_355725573.ann
Error in file:  09112R_619440406.ann
Error in file:  16263R_359218191.ann
Error in file:  17677R_354095974.ann
Error in file:  17677R_354095974.ann
Error in file:  S0141933115000630.ann
Error in file:  S0141933115000630.ann
Error in file:  S0141933115000630.ann
Error in file:  S0165011412002217.ann
Error in file:  S0166093415003067.ann
Error in file:  S0166218X1400016X.ann
Error in file:  S0168192316303598.ann
Error in file:  S0168192316303598.ann
Error in file:  S0169260714002946.ann
Error in file:  S0267364914001277.ann
Error in file:  S0268005X16300492.ann
Error in file:  S0301679X1500242X.ann
Error in file:  S0304397511003926.ann
Error in file:  S03043975

In [5]:
########PROCESS TRAINING SET##########
res = tokenize_with_bert(train,tokenizer)
train['Sentence_Tokenized'] = res[0]
train['Token_Spans'] = res[1]

#Get Gold span tags for organizations
train['Gold_Span_Tags_ORG_IOB'] = get_span_tags_iob(train,org_dict_gold,"","Token_Spans")
#Get Gold span tags for grants
train['Gold_Span_Tags_GRT_IOB'] = get_span_tags_iob(train,grant_dict_gold,"","Token_Spans")
#Fix the tags
train['Gold_Span_Tags_ORG_IOB'] = fix_iob_tags(train['Gold_Span_Tags_ORG_IOB'].values)
train['Gold_Span_Tags_GRT_IOB'] = fix_iob_tags(train['Gold_Span_Tags_GRT_IOB'].values)
#Merge the tags
train['Gold_Span_Tags_IOB'] = merge_grant_and_org_annotations_iob(train,'Gold_Span_Tags_GRT_IOB','Gold_Span_Tags_ORG_IOB')

#Separate a part for monitoring the training
train_monitor_idx = np.random.choice(train[train.Folder=='GoldSet_2019\\KPI'].Filename.unique(),1000,replace=False)
train_monitor = train[train.Filename.isin(train_monitor_idx)].reset_index(drop=True).copy(deep=True)
train = train[~train.Filename.isin(train_monitor_idx)].reset_index(drop=True).copy(deep=True)

Length of input df:  27416
Number of rows with conflicting annotations:  33
Percentage:  0.12036766851473593 %


In [6]:
########PROCESS VALIDATION SET##########
res = tokenize_with_bert(valid,tokenizer)
valid['Sentence_Tokenized'] = res[0]
valid['Token_Spans'] = res[1]

#Get Gold span tags for organizations
valid['Gold_Span_Tags_ORG_IOB'] = get_span_tags_iob(valid,org_dict_gold,"","Token_Spans")
#Get Gold span tags for grants
valid['Gold_Span_Tags_GRT_IOB'] = get_span_tags_iob(valid,grant_dict_gold,"","Token_Spans")
#Fix the tags
valid['Gold_Span_Tags_ORG_IOB'] = fix_iob_tags(valid['Gold_Span_Tags_ORG_IOB'].values)
valid['Gold_Span_Tags_GRT_IOB'] = fix_iob_tags(valid['Gold_Span_Tags_GRT_IOB'].values)
#Merge the tags
valid['Gold_Span_Tags_IOB'] = merge_grant_and_org_annotations_iob(valid,'Gold_Span_Tags_GRT_IOB','Gold_Span_Tags_ORG_IOB')

Length of input df:  5012
Number of rows with conflicting annotations:  3
Percentage:  0.05985634477254589 %


In [ ]:
#####WRITE TO FILE#########
with open(".\\Corpus\\training.txt",'w',encoding='utf-8') as f:
    for index, row in train.iterrows():
        if len(row['Sentence'])<=1000:
            words = row['Sentence_Tokenized']
            tokens = row['Gold_Span_Tags_IOB']
            for i in range(len(tokens)):
                f.write(words[i]+" "+tokens[i]+'\n')
            f.write('\n')
with open(".\\Corpus\\validation.txt",'w',encoding='utf-8') as f:
    for index, row in train_monitor.iterrows():
        if len(row['Sentence'])<=1000:
            words = row['Sentence_Tokenized']
            tokens = row['Gold_Span_Tags_IOB']
            for i in range(len(tokens)):
                f.write(words[i]+" "+tokens[i]+'\n')
            f.write('\n')
with open(".\\Corpus\\test.txt",'w',encoding='utf-8') as f:
    for index, row in valid.iterrows():
        words = row['Sentence_Tokenized']
        tokens = row['Gold_Span_Tags_IOB']
        for i in range(len(tokens)):
            f.write(words[i]+" "+tokens[i]+'\n')
        f.write('\n')

In [ ]:
with open("flair_valid.pkl",'wb') as f:
    pickle.dump(valid,f)